#Тест скорости инференса
В данном ноутбуке выполняется замер скорости инференса модели с рандомными весами.
Для этого скачивается проект, собирается модель, загружается тестовый набор изображений, выполняется предобработка одного из выбранных избражений и наконец производиться замер времени инференса.

In [1]:
import sys
import os
import gdown
import numpy as np

In [2]:
# Устанавливаем модуль import_ipynb и импортируем его.
# Он необходим для импорта файлов .ipynb.
try:
    import import_ipynb
except:
    ! pip install import-ipynb
    import import_ipynb

# Если проект отсутствует, клонируем его с Github. Далее из проекта нам понадобиться файл 
# модели - model.ipynb, расположенный в корне проекта.
# Импортируем из него функцию сборки модели
try:
    from skillbox_thesis_project.model import assemble_model
except ModuleNotFoundError:
    ! git clone https://github.com/GrinkoL/skillbox_thesis_project.git
    from skillbox_thesis_project.model import assemble_model
    
# Cобираем модель
model = assemble_model()

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=cd0443b46b294a0975d7b83b6d2bacb6f5cfd71e6a4c8728f0875e560d0f144c
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Cloning into 'skillbox_thesis_project'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 202 (delta 18), reused 0 (delta 0), pack-reused 159
Receiving objects: 100% (202/202), 768.45 KiB | 28.46 MiB/s, done.
Resolving deltas: 100% (70/70), done.
importing Jupyter notebook from /content/skillbox_thesis_project/model.ipynb
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-u89x587_
580075520/580070376 [==============================] - 6s 0us/step


In [3]:
# Загружаем и распаковываем тестовый датасет, а также удаляем загруженный архив.
# Из него мы возьмём одно изображения, на котором проверим скорость инференса.
if not os.path.exists('test'):
    if not os.path.exists('test.zip'):
        gdown.download('https://drive.google.com/uc?id=1bGHeWeWYXj5biL9s-qTc9gyv91WNAbWE', 'test.zip', False)
    ! unzip -j test.zip -d test > /dev/null
    ! rm test.zip

Downloading...
From: https://drive.google.com/uc?id=1bGHeWeWYXj5biL9s-qTc9gyv91WNAbWE
To: /content/test.zip
222MB [00:01, 136MB/s]


In [4]:
from tensorflow.keras import backend as K

#  Определяем функцию предобработки изображения
def preprocess_input(path_img):
        img = image.load_img(path_img, target_size=(224,224))
        x_temp = image.img_to_array(img)
        data_format = K.image_data_format()
        assert data_format in {'channels_last', 'channels_first'}

        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 93.5940
            x_temp[:, 1, :, :] -= 104.7624
            x_temp[:, 2, :, :] -= 129.1863
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 93.5940
            x_temp[..., 1] -= 104.7624
            x_temp[..., 2] -= 129.1863

        return x_temp[None,...]

In [5]:
from tensorflow.keras.preprocessing import image

img_path = 'test/0.jpg'
img = preprocess_input(img_path)

In [7]:
# ВНИМАНИЕ! Первый запуск инференса с аппаратным ускорителем происходит долго,
# порядка нескольких секунд, поэтому замерять нужно со второго запуска.

%%time
np.argmax(model(img))

CPU times: user 16.5 ms, sys: 3.97 ms, total: 20.4 ms
Wall time: 21.7 ms


5

#Вывод:
Созданная нами модель имеет скорость инференса, равную ~ 22ms на одном изображении. Согласно техзаданию к дипломному проекту, скорость инференса не должна превышать 330 ms, у нас же она на порядок меньше.